In [1]:
# things we need for NLP
# NLTK is a leading platform for building Python programs to work with human language data.
# It provides easy-to-use interfaces to over 50 corpora and lexical resources such as WordNet,
# along with a suite of text processing libraries for classification, tokenization, stemming, tagging, parsing,
# and semantic reasoning.
import nltk
# A computer program or subroutine that stems word may be called a stemming program, stemming algorithm, or stemmer.
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
# things we need for Tensorflow
# Numpy is a library for the Python programming language, adding 
# support for large, multi-dimensional arrays and matrices, along with a 
# large collection of high-level mathematical functions thttp://localhost:8888/notebooks/Documents/Notes/cg%20chatty%202/cg%20chatbotty/cg%20chatty.ipynb#o operate on these arrays.
import numpy as np
# TFlearn is a modular and transparent deep learning library built on top of Tensorflow.
# It was designed to provide a higher-level API to TensorFlow in order to facilitate and speed-up experimentations,
# while remaining fully transparent and compatible with it.
import tflearn
# Tensorflow is an open source machine learning library, which is used to develop and train ML models
import tensorflow as tf
import random

W0829 11:53:12.843653  8180 deprecation_wrapper.py:119] From c:\program files\python37\lib\site-packages\tflearn\helpers\summarizer.py:9: The name tf.summary.merge is deprecated. Please use tf.compat.v1.summary.merge instead.

W0829 11:53:12.848667  8180 deprecation_wrapper.py:119] From c:\program files\python37\lib\site-packages\tflearn\helpers\trainer.py:25: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.

W0829 11:53:12.861199  8180 deprecation_wrapper.py:119] From c:\program files\python37\lib\site-packages\tflearn\collections.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0829 11:53:12.868218  8180 deprecation_wrapper.py:119] From c:\program files\python37\lib\site-packages\tflearn\config.py:123: The name tf.get_collection is deprecated. Please use tf.compat.v1.get_collection instead.

W0829 11:53:12.880250  8180 deprecation_wrapper.py:119] From c:\program files\python37\lib\site-packages\t

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
# import our chat-bot intents file
import json
intent_file= open('intents.json','r')
intents = json.load(intent_file)
intent_file.close()

In [3]:
words = []
classes = []
references = []
words_to_ignore = ['?','!']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    #note arr['name'] is visiting that array by name 
    for pattern in intent['question_patterns']:
        # add to list containing the classes of words
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
        # tokenize each word in the sentence
#Tokenization is the act of breaking up a sequence of strings into pieces such as words, keywords, phrases, symbols and other elements called tokens
        word = nltk.word_tokenize(pattern)
        # append word to our words list
        words.extend(word)
        # add to documents in our corpus 
#         A corpus is a collection of machine-readable texts that have been produced in a natural communicative setting
#in the references multidim list of bot, append a list of words at index 0 and a list of contents of intents with key 'tag'
        references.append((word, intent['tag']))
        

# stem and lower each word and remove duplicates
words = [ stemmer.stem(word.lower()) for word in words if word not in words_to_ignore]
#set method removes the duplicates in the words and classes obtained
words = list(set(words))
classes = list(set(classes))

print (len(references), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

131 documents
28 classes ['EntSolutions', 'get_services', 'soft_issues_response_soft_antivirus', 'departments', 'soft_issues_response_noantivirus', 'bug_per', 'fins_trans', 'hours', 'soft_issues_response_errors', 'thanks', 'opentoday', 'ent_trans', 'finfinancials', 'management_structure', 'soft_issues', 'soft_issues_response_hard', 'payments', 'bug_resolve_affirmed', 'transaction', 'payments_n_clearing', 'location_question', 'get_info', 'services', 'Ebanking', 'greeting', 'e_trans', 'goodbye', 'clearing_trans']
116 unique stemmed words ['posit', 'is', 'problem', 'yep', 'goodby', 'fintech', 'yo', 'no', 'deal', 'sort', 'on', 'a', 'clos', 'expery', 'hour', 'can', 'anyon', 'much', 'exceiv', 'phys', 'good', 'let', 'mor', 'the', 'er', 'tour', 'for', 'hav', 'that', 'tel', 'bug', 'man', 'tak', 'field', 'out', 'yeah', 'program', 'visit', 'loc', 'i', 'to', 'up', 'find', 'nee', 'hierachy', 'what', 'issu', 'depart', 'help', 'provid', 'am', 'ok', 'op', 'who', 'consult', 'ther', 'solv', 'clear', 'se

In [4]:
# create our training data array bags
training = []
output = []
# create an empty array for our output
null_output = [0] * len(classes)
#A vocabulary is a set of all words known to our bot
# training set, vocabulary of words for each sentence
#In the following method, we will convert the list of words we have to an array of tensors that tensorflow can use
for ref in references:
    # initialize our vocabulary of words
    vocab = []
    # list of tokenized words for the pattern
    pattern_words = ref[0]
    # stem each word, to its root(s) :-)
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our vocabulary of words array
    for w in words:
        vocab.append(1) if w in pattern_words else vocab.append(0)

    # output is a '0' for inactive and '1' for active tag
    output_row = list(null_output)
    output_row[classes.index(ref[1])] = 1
#append to multidim array training the vocabulary of root forms and the row they each correspond to
    training.append([vocab, output_row])

# shuffle our features and turn into np.array
#shuffling will reduce the variance and overfitting
random.shuffle(training)
training = np.array(training)

# create train and test lists
#the : stands for everything from the beginning to end , 0 represents the dimension, the 1st in 1d array
train_x = list(training[:,0])
train_y = list(training[:,1])
#print(train_x)

In [23]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
#shape will be 2D array of 1*length of a single train list
input_data = tflearn.input_data(shape=[None, len(train_x[0])])
#create fully connected layer with 16 neurons
#  6 is an arbitrary value chosen according to the number of bits you want to 
#be able to compress your network's trained parameters into.
layer1 = tflearn.fully_connected(input_data,16,activation='ReLU')
layer2 = tflearn.fully_connected(layer1,16,activation='relu6')
net = tflearn.fully_connected(layer1, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=500, batch_size=3, show_metric=True,validation_batch_size=0.3)
model.save('model.tflearn')


Training Step: 21999  | total loss: 0.34411 | time: 0.069s
| Adam | epoch: 500 | loss: 0.34411 - acc: 0.8318 -- iter: 129/131
Training Step: 22000  | total loss: 0.30971 | time: 0.071s
| Adam | epoch: 500 | loss: 0.30971 - acc: 0.8487 -- iter: 131/131
--


In [6]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [7]:

trained_data = pickle.load( open( "training_data", "rb" ) )
words = trained_data['words']
classes = trained_data['classes']
train_x = trained_data['train_x']
train_y = trained_data['train_y']

In [8]:
# load our saved model
model.load('./model.tflearn')


W0829 11:53:14.019701  8180 deprecation.py:323] From c:\program files\python37\lib\site-packages\tensorflow\python\training\saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [9]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def make_vocab(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    #vocabulary of words
    vocab = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                vocab[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(vocab))

In [10]:
p = make_vocab("is your shop open today?", words)
print (p)

[0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0]


In [11]:
# create a data structure to hold user context
context = {}
#minimum probability we accept that the bot is sure of what it says, in this case bot must be at least 
#10% sure before it replies
ERROR_THRESHOLD = 0.60
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([make_vocab(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[k,v] for k,v in enumerate(results) if v>ERROR_THRESHOLD]
    # sort by strength of probability/descending order
    results.sort(key=lambda x: x[1], reverse=True)
    classify_result = []
    for result in results:
        classify_result.append((classes[result[0]], result[1]))
    # return tuple of intent and probability
    return classify_result

def response(sentence, userID='user', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for intent in intents['intents']:
                # find a tag matching the first result
                if intent['tag'] == results[0][0]:
                   #if the array json object contains a value of 'context_set', set that to the user session
                    if 'context_set' in intent:
                        if show_details: print ('context:', intent['context_set'])
                        context[userID] = intent['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in intent or \
                        (userID in context and 'context_filter' in intent and intent['context_filter'] == context[userID]):
                        if show_details: print ('tag:', intent['tag'])
                        # a random response from the intent
                        return str(random.choice(intent['responses']))

            results.pop(0)

In [12]:
context

{}

In [13]:
response(" there!", show_details=True)

In [14]:
response('today')

In [15]:
classify('whiii doa youd consult d ')

[]

In [16]:
response('hi')

In [17]:
response('i have some erors')

'please read the log message displayed on the package console: is it ending with HARD or SOFT?'

In [18]:
classify("explain finfinancials?")

[('soft_issues_response_hard', 0.9999534)]

In [19]:
context

{'user': 'error_finder'}

In [ ]:
# Flask is a micro framework for Python aiming to provide the flexibility of creating web sites and APIs.
from flask import Flask, request,render_template
from flask_cors import CORS

app = Flask(__name__)
app.static_folder = 'static'
#use the cors module to allow unknown sources to access the process in the server's ip and get responses. This obvously needs
#some alteration for security purposes
CORS(app)
#default route
@app.route("/") 
def home():
    return render_template("home.html") 

@app.route('/process') 
def get_bot_response():
    userText = request.args['message'] 
    bot_response=response(userText) 
    if(bot_response is None): 
        bot_response=random.choice(['Sorry, I fail to understand that, Im a bot','Please rephrase the question, im just a bot']) 
    return bot_response 

#run flask session on this host
if __name__ == '__main__':
    app.run(host='10.100.0.177',port=8081)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


I0829 15:44:51.420146  8180 _internal.py:122]  * Running on http://10.100.0.177:8081/ (Press CTRL+C to quit)
I0829 15:44:52.899079  1524 _internal.py:122] 10.100.0.177 - - [29/Aug/2019 15:44:52] "GET / HTTP/1.1" 200 -
I0829 15:44:52.932669 16180 _internal.py:122] 10.100.0.177 - - [29/Aug/2019 15:44:52] "GET /static/static/speech-balloon.svg HTTP/1.1" 404 -


In [26]:
# Flask is a micro framework for Python aiming to provide the flexibility of creating web sites and APIs.
from flask import Flask, request,render_template
app = Flask(__name__)
app.static_folder = 'static'
@app.route('/')
def index():
	return render_template('index.html')
@app.route('/process',methods=['post'])
def process():
	user_input=request.form['user_input']
	bot_response=response(user_input)
	if(bot_response is None):
		bot_response=random.choice(['Sorry, I fail to understand that, Im a bot','Please rephrase the question, im just a bot'])
    



	return render_template('index.html',user_input=user_input,bot_response=bot_response)
# if __name__ == '__main__':
# 	app.run()
# #make the session last longer as jupyter does not provide the ability
from werkzeug.serving import run_simple
run_simple('localhost', 9000, app)

I0829 12:10:02.565681  8180 _internal.py:122]  * Running on http://localhost:9000/ (Press CTRL+C to quit)


In [22]:
from django.shortcuts import render, render_to_response
from django.http import HttpResponse
from django.views.decorators.csrf import csrf_exempt

import asyncio
import websockets
# Train based on the english corpus

#Already trained and it's supposed to be persistent
#chatbot.train("chatterbot.corpus.english")

async def chat_receiver(websocket, path):
    async for message in websocket:
        message = json.loads(message)
        text = message['text']

        chat_response = chatbot.get_response(text).text
        print(chat_response)
        await websocket.send(json.dumps({'response': chat_response}))

async def router(websocket, path):
	if path == "/":
		await chat_receiver(websocket, path)

    #the code below is how you add other path's
    
	# elif path == "/shade-area":
	# 	await get_shaded_area(websocket, path)
	# elif path == '/render':
	# 	await render(websocket, path)

asyncio.get_event_loop().run_until_complete(websockets.serve(router, '0.0.0.0', 8765))

asyncio.get_event_loop().run_forever()

RuntimeError: This event loop is already running

In [ ]:
from django.shortcuts import render, render_to_response
from django.http import HttpResponse
import json
from django.views.decorators.csrf import csrf_exempt
from django.conf import settings
settings.configure()


# Train based on the english corpus

#Already trained and it's supposed to be persistent
#chatbot.train("chatterbot.corpus.english")

@csrf_exempt
def get_response(request):
	response = {'status': None}

	if request.method == 'POST':
		data = json.loads(request.body)
		message = data['message']

		chat_response = "that's fine"
		response['message'] = {'text': chat_response, 'user': False, 'chat_bot': True}
		response['status'] = 'ok'

	else:
		response['error'] = 'no post data found'

	return HttpResponse(
		json.dumps(response),
			content_type="application/json"
		)


def home(request, template_name="home.html"):
	context = {'title': 'Chatbot Version 1.0'}
	return render_to_response(template_name, context)
home('hi')